In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import f1_score
from joblib import dump, load

In [ ]:
df1 = pd.read_pickle("projects_16_18_essay_features_only.pkl")

In [ ]:
df2 = pd.read_parquet("tokens_single_file_part-00000-5bdd9372-029e-4ce4-869e-12a302797c34-c000.snappy.parquet")

In [ ]:
df3 = pd.read_pickle("other_feats.pkl")

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.shape, df2.shape, df3.shape

In [ ]:
df = df1.merge(df3, left_on = 'Project ID', right_on = 'Project ID').merge(df2, left_on = 'Project ID', right_on = 'ProjectID')
df = df[df['Project Current Status'] != 'Live']

In [ ]:
df['Project Current Status'].value_counts()

In [ ]:
df['Project Current Status Coded'] = df['Project Current Status'].apply(lambda x: int(x == 'Fully Funded'))

In [ ]:
df['Project Current Status Coded'].value_counts(normalize=True)

In [ ]:
del df['Project Current Status']
del df['ProjectID']
del df['Project ID']
df.shape

In [ ]:
df.head()

In [ ]:
Y = df['Project Current Status Coded']
X = df
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

In [ ]:
df_fully_funded = X_train[X_train['Project Current Status Coded'] == 1]
df_expired = X_train[X_train['Project Current Status Coded'] == 0]
df_fully_funded = resample(df_fully_funded, n_samples=df_expired.shape[0], replace=False, random_state=0)
df_balance = df_fully_funded.append(df_expired)

In [ ]:
y_train = df_balance['Project Current Status Coded']
X_train = df_balance
del X_train['Project Current Status Coded']
del X_test['Project Current Status Coded']

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
# clf = RandomForestClassifier(random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
len(df2.columns)

In [ ]:
dump(clf, 'logit.joblib')

In [ ]:
clf = load('logit.joblib')

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.coef_